In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import random
import os

# Basic knowledge: 
To begin this project, it would be good to hold a minimum understanding of the subject we will be analyzing. As I did not know much about this topic at the day the project started, I have recurred to the shark-attack wiki: https://en.wikipedia.org/wiki/Shark_attack



# Defining the dataset path, and importing it to begin basic dataset exploration

Questions: 
- how to add the dataset to gitignore?

In [2]:
# To follow along and access the DataSet, download it from KAGGLE using this link
# https://www.kaggle.com/teajay/global-shark-attacks

# Once you have downloaded the DataSet, change the dataset variable to match the 
# path where you have saved the 'attacks.csv' file.
dataset = 'attacks.csv' 
df = pd.read_csv(dataset, encoding='latin-1')

Now, we will check some basic information about the dataset, in order to formulate a more educated hypothesis which we could actually put to test with the data available.

In [3]:
display(df.shape)# To know the shape of the DF
print(df.drop_duplicates().shape) # Shape when eliminating duplicates

(25723, 24)

(6312, 24)


Here, I notice that the shape of the df with no duplicates is very small when compared to the whole df. This seems weird and since the `drop_duplicates` function ignores time indexes, I'll try to compare both dataframes' time data

In [4]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
df.Date

0        25-Jun-2018
1        18-Jun-2018
2        09-Jun-2018
3        08-Jun-2018
4        04-Jun-2018
            ...     
25718            NaN
25719            NaN
25720            NaN
25721            NaN
25722            NaN
Name: Date, Length: 25723, dtype: object

In [6]:
df_nodupes = df.drop_duplicates()
df_nodupes.Date

0        25-Jun-2018
1        18-Jun-2018
2        09-Jun-2018
3        08-Jun-2018
4        04-Jun-2018
            ...     
6307             NaN
6308             NaN
6309             NaN
8702             NaN
25722            NaN
Name: Date, Length: 6312, dtype: object

In [7]:
# Now, with a df smaller in size, I want to see what info is there on the last couple
# of columns which have unexplicit
df_nodupes[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']]

,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...
6307,NaN,NaN,6309.0,NaN,NaN
6308,NaN,NaN,6310.0,NaN,NaN
6309,NaN,NaN,NaN,NaN,NaN
8702,NaN,NaN,NaN,NaN,NaN


In [8]:
# Too many null values... let's count them and let
print(df_nodupes.shape)
df_nodupes[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']].isnull().sum()

(6312, 24)


Case Number.1       10
Case Number.2       10
original order       3
Unnamed: 22       6311
Unnamed: 23       6310
dtype: int64

In [9]:
# If only 1 value in the 'Unnamed: 22' column, and 2 values in the
# 'Unnamed: 22' column, I'll not consider this data for my analysis.
df_nodupes = df_nodupes.drop(columns=['Unnamed: 22', 'Unnamed: 23'])

In [10]:
# Now we'll look at the columns again
df_nodupes.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [11]:
# The following columns seemed a little bit rare, so i do a value count to find out what they are about
df_nodupes['Case Number.1'].value_counts()

1952.08.04      2
1913.08.27.R    2
1962.06.11.b    2
2006.09.02      2
1990.05.10      2
               ..
1860.08.01      1
1978.09.16      1
1981.10.19.a    1
2007.06.30.b    1
1965.03.23      1
Name: Case Number.1, Length: 6285, dtype: int64

In [12]:
df_nodupes['Case Number.2'].value_counts()

2005.04.06        2
1907.10.16.R      2
1915.07.06.a.R    2
1913.08.27.R      2
1983.06.15        2
                 ..
1919.01.05        1
1973.09.09        1
1860.08.01        1
1978.09.16        1
1965.03.23        1
Name: Case Number.2, Length: 6286, dtype: int64

In [13]:
df_nodupes['original order'].value_counts()

569.0     2
4603.0    1
4899.0    1
810.0     1
796.0     1
         ..
3508.0    1
3256.0    1
3106.0    1
3080.0    1
6272.0    1
Name: original order, Length: 6308, dtype: int64

In [14]:
#Since 'original order seems like arbitrary indexes, i'll drop it
df_nodupes = df_nodupes.drop(columns='original order')
df_nodupes.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2'],
      dtype='object')

In [15]:
# I want to check out some of the pdf and href
print(df_nodupes[['pdf', 'href']].isnull().sum())
df_nodupes[['pdf', 'href']]

pdf     10
href    10
dtype: int64


,pdf,href
0,2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...
6307,NaN,NaN
6308,NaN,NaN
6309,NaN,NaN
8702,NaN,NaN


In [16]:
# I want to check what is on those pdfs
# A random sample of the column 

# With a FOR loop
for i in range(10):
    e = random.choice(range(1000))
    print(f"index: {e}, link: {df_nodupes.iloc[e]['href']}")

# With Random sample
display(random.sample(list(df_nodupes['href']), 10))

# Below are a couple of the links, when I open them, I have found that they are seem quite structured
# It could be possible to parse them later down the road and use a REGEX to find more data

# I also have ran this column a few times to notice that all pdfs have actually been uploaded to
# the same website and have the same naming structure

index: 422, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2015.05.24-Wheatro.pdf
index: 825, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2012.01.15-Msungubana.pdf
index: 321, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2015.12.25-GrandCanary.pdf
index: 600, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2013.11.22-Gardiner
index: 711, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2013.01.05-Swaffer.pdf
index: 442, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2015.03.16-Bora-Bora.pdf
index: 113, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2017.07.23.a-Majorca.pdf
index: 370, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2015.09.00-Lautiki.pdf
index: 220, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2016.09.07-Bagnol.pdf
index: 255, link: http://sharkattackfile.net/spreadsheets/pdf_directory/2016.06.27-Sullivans.pdf


['http://sharkattackfile.net/spreadsheets/pdf_directory/2018.04.15.b-Longrass.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/2007.06.30.a-Zinner.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1864.00.00.R-Sumatra.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1997.08.02.b-Cesar.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1997.05.31.c-Medvec.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1932.09.26.R-Fiji.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1959.05.16-Goldback.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1850.00.00-LakeNicaragua.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/1960.07.03-Smith.pdf',
 'http://sharkattackfile.net/spreadsheets/pdf_directory/2000.09.15-Smith.pdf']

# After some random sampling, I've noticed that the following indexes have some mistakes.. 
# a REGEX can be used to fix problems like these 


In [17]:
print(df_nodupes.iloc[332]['href'])
print(df_nodupes.iloc[324]['href'])
print(df_nodupes.iloc[588]['href'])
print(df_nodupes.iloc[569]['href'])

http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2015.11.15.a-Engelman.pdf
http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2015.12.21.a-Brazil.pdf
http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2014.00.00.b-OceanicWhitetip.pdf
http://sharkattackfile.net/spreadsheets/pdf_directory/http://sharkattackfile.net/spreadsheets/pdf_directory/2014.04.03-Armstrong.pdf


## It looks still like some of these pdfs are duplicates, even after dropping duplicates :

In [18]:
# how many times each pdf on the dataframe
df_pdf = df_nodupes["pdf"]
df_pdf.value_counts()

1929.03.04.a-b.Roads-Aldridge.pdf      2
1916.07.12.a-b-Stillwell-Fisher.pdf    2
1931.09.21.a-b-Holaday-Barrows.pdf     2
1907.10.16.R-HongKong.pdf              2
1921.11.27.a-b-Jack.pdf                2
                                      ..
1983.10.17.a-Kalein.pdf                1
1992.01.06-Mauritius.pdf               1
1944.10.25.b-Carter.pdf                1
2017.06.10.b-Flinders.pdf              1
1935.01.24.a-Flat-bottomed-boat.pdf    1
Name: pdf, Length: 6291, dtype: int64

In [19]:
# drop dupes and compare lengths
df_pdf_nodupes = df_pdf.drop_duplicates()

len(df_pdf) - len(df_pdf_nodupes), 'duped values'

(20, 'duped values')

## Since the lengths are not the same, I will check if those duplicated entries are only in this column

In [20]:
# there are 20 duplicated values on the pdf columns
df_nodupes.duplicated('pdf').value_counts()

False    6292
True       20
dtype: int64

In [21]:
# But only 18 dupes if we take Location into count
df_nodupes.duplicated(['pdf','Location']).value_counts()

False    6294
True       18
dtype: int64

### I'll look at the rest of the data now.

In [22]:
print(df_nodupes.shape)
df_nodupes.duplicated().sum()

(6312, 21)


7

In [23]:
df_nodupes = df_nodupes.drop_duplicates()
df_nodupes.duplicated().sum()

0

In [24]:
print(df_nodupes.shape)
df_nodupes[["Date", "Location", "pdf"]]

(6305, 21)


,Date,Location,pdf
0,25-Jun-2018,"Oceanside, San Diego County",2018.06.25-Wolfe.pdf
1,18-Jun-2018,"St. Simon Island, Glynn County",2018.06.18-McNeely.pdf
2,09-Jun-2018,"Habush, Oahu",2018.06.09-Denges.pdf
3,08-Jun-2018,Arrawarra Headland,2018.06.08-Arrawarra.pdf
4,04-Jun-2018,La Ticla,2018.06.04-Ramos.pdf
...,...,...,...
6300,1883-1889,"Panama Bay 8ºN, 79ºW",ND-0002-JulesPatterson.pdf
6301,1845-1853,"Below the English fort, Trincomalee",ND-0001-Ceylon.pdf
6302,NaN,NaN,NaN
8702,NaN,NaN,NaN


In [25]:
df_nodupes.Country.value_counts()

USA                         2229
AUSTRALIA                   1338
SOUTH AFRICA                 579
PAPUA NEW GUINEA             134
NEW ZEALAND                  128
                            ... 
CYPRUS                         1
BRITISH ISLES                  1
JAVA                           1
TASMAN SEA                     1
NORTHERN MARIANA ISLANDS       1
Name: Country, Length: 212, dtype: int64

In [26]:
# While checking the columns 'Species ' and 'Sex ' have unnecesary spaces at the end of the string
# to remove these, and also take out the '(Y/N)' from the column 'Fatal'

In [27]:
df_label = df_nodupes
df_label.columns = ['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time',
       'Species', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2']
df_label.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex', 'Age', 'Injury', 'Fatal', 'Time', 'Species',
       'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2'],
      dtype='object')

In [28]:
df_label['Fatal'].value_counts()

N          4293
Y          1388
UNKNOWN      71
 N            7
y             1
N             1
M             1
2017          1
Name: Fatal, dtype: int64

In [29]:
remove_spaces = lambda x:  x.remove(' ') if ' ' in x else x

df_label['Fatal'] = list(
                            map(remove_spaces(
                            df_label['Fatal']),
                            ))
    
df_label['Fatal'].value_counts()


# I want to see the indexes which have a duplicated pdf row
dupes = []
for a,b in list(df_label['pdf'].duplicated().items()):
    if b:
        dupes.append(a)
dupes

TypeError: map() must have at least two arguments.

In [ ]:
df_label.loc[dupes]

In [ ]:
dfx = df_nodupes["pdf"].value_counts() if 

In [ ]:
# @@ Use this to fill null values: 
# df_clean["drive"] = df_clean.drive.fillna("NoTransmision")

In [ ]:
display(df.columns) # To know which are the columns in the DF
display(df.count()) # To know how much data are we missin on each column
display(df.dtypes)